In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import cv2
from sklearn.model_selection import train_test_split
from itertools import product
from PIL import Image
import glob
import zipfile

import torch
import torchvision
from torchvision import models,transforms,datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,ConcatDataset


In [ ]:
# Data path
data_dir = '../input/aerial-cactus-identification/'
out_dir = './'

train_path = '../kaggle/temp/train/'
test_path = '../kaggle/temp/test/'


In [ ]:
with zipfile.ZipFile(data_dir + "train.zip","r") as z:
    z.extractall("../kaggle/temp/")
    
with zipfile.ZipFile(data_dir + "test.zip","r") as z:
    z.extractall("../kaggle/temp/")


In [ ]:
print('Num train samples:{0}'.format(len(os.listdir(train_path))))
print('Num test samples:{0}'.format(len(os.listdir(test_path))))


In [ ]:
labels = pd.read_csv(data_dir + 'train.csv')
sub = pd.read_csv(data_dir + 'sample_submission.csv')


In [ ]:
labels.head()


In [ ]:
labels.info()


In [ ]:
num_cactus = labels[labels['has_cactus']==1]['id'].count()
num_no_cactus = labels[labels['has_cactus']==0]['id'].count()


In [ ]:
tags = 'Cactus', 'No cactus'
sizes = [num_cactus, num_no_cactus]
explode = (0, 0.1)  # "explode" the 2nd slice

fig, ax = plt.subplots()
ax.pie(sizes, explode=explode, labels=tags, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.title("Number of images with/without cactus")
plt.show()


In [ ]:
fig,ax = plt.subplots(1,5,figsize=(15,3))

for i, idx in enumerate(labels['id'][-5:]):
    path = os.path.join(train_path,idx)
    ax[i].imshow(cv2.imread(path)) # [...,[2,1,0]]


In [ ]:
# Hyper parameters
num_epochs = 20
num_classes = 2
batch_size = 32
learning_rate = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [ ]:
train, val = train_test_split(labels, stratify=labels.has_cactus, test_size=0.2)
train.shape, val.shape, labels.shape


In [ ]:
class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name)
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label


In [ ]:
# Image preprocessing
trans_train = transforms.Compose([transforms.ToPILImage(),
#                                   transforms.Pad(32, padding_mode='reflect'),
#                                   transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]),
                                  transforms.ToTensor()])

trans_valid = transforms.Compose([transforms.ToPILImage(),
#                                   transforms.Pad(32, padding_mode='reflect'),
#                                   transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5]),
                                  transforms.ToTensor()])

# Data generators
dataset_train = MyDataset(df_data=train, data_dir=train_path, transform=trans_train)
dataset_valid = MyDataset(df_data=val, data_dir=train_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size, shuffle=False)


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
#         self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
#         self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
#         self.bn4 = nn.BatchNorm2d(256)
#         self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg = nn.AvgPool2d(4)
        self.fc = nn.Linear(128, 1)
        self.out = nn.Sigmoid()
   
    def forward(self, x):
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x)))) # first convolutional layer then batchnorm, then activation then pooling layer.
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
#         x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
#         x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
#         print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 128) # !!!
        x = self.fc(x)
        x = self.out(x)
        return x


In [ ]:
model = SimpleCNN().to(device)


In [ ]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))


In [ ]:
print(targets.dtype)
new_targets = targets.float()
print(new_targets.dtype)
print(outputs.dtype)


In [ ]:
# Train the model
total_step = len(loader_train)

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(loader_train):
        targets = targets.float()
        
        images = images.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.data, targets)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


In [ ]:
# Test the model
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, targets in loader_valid:
        predicted = []
        targets = targets.float()
        images = images.to(device)
        targets = targets.to(device)
        outputs = model(images)
#         print(outputs)
        
        for out in outputs.data:
            if out < 0.5:
                predicted.append(0)
            else:
                predicted.append(1)
                
        predicted = torch.FloatTensor(predicted).to(device)

        total += targets.size(0)
        correct += (predicted == targets).sum()
          
    print('Test Accuracy of the model on the 1750 validation images: {} %'.format(100 * correct / total))


In [ ]:
# generator for test data 
dataset_valid = MyDataset(df_data=sub, data_dir=test_path, transform=trans_valid)
loader_test = DataLoader(dataset = dataset_valid, batch_size=1, shuffle=False)


In [ ]:
model.eval()

preds = []
for batch_i, (data, target) in enumerate(loader_test):
    data, target = data.cuda(), target.cuda()
    output = model(data)
    
    for out in output.data:
        if out < 0.5:
            preds.append(0)
        else:
            preds.append(1)

sub['has_cactus'] = preds
sub.to_csv('submission.csv', index=False)
